In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob # This allows all varibles to be global??
import os # 
from pprint import pprint # what is pprint?
import simplejson as json
import requests
import time
from IPython.display import display

In [2]:
URI = "myucdsqlserver.c1d6wbdesyas.eu-west-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbikes"
USER = "danny"
PASSWORD = "panda1989"

In [3]:
engine = create_engine("mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [ ]:
# Creation of both tables, Stations and Availability. Code taken from the lecture notes

metadata = sqla.MetaData()
station = sqla.Table("station", metadata,
    sqla.Column('address', sqla.String(256), nullable=False),
    sqla.Column('banking', sqla.Integer),
    sqla.Column('bike_stands', sqla.Integer),
    sqla.Column('bonus', sqla.Integer),
    sqla.Column('contract_name', sqla.String(256)),
    sqla.Column('name', sqla.String(256)),
    sqla.Column('number', sqla.Integer),
    sqla.Column('position_lat', sqla.REAL),
    sqla.Column('position_lng', sqla.REAL),
    sqla.Column('status', sqla.String(256))
)

availability = sqla.Table("availability", metadata,
    sqla.Column('available_bikes', sqla.Integer),
    sqla.Column('available_bike_stands', sqla.Integer),
    sqla.Column('number', sqla.Integer),
    sqla.Column('last_update', sqla.BigInteger)
)

try:
    station.drop(engine)
    availability.drop(engine)
except:
    pass

metadata.create_all(engine)

2017-04-23 16:10:35,743 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-04-23 16:10:35,745 INFO sqlalchemy.engine.base.Engine {}
2017-04-23 16:10:35,800 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-04-23 16:10:35,803 INFO sqlalchemy.engine.base.Engine {}
2017-04-23 16:10:35,867 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-04-23 16:10:35,869 INFO sqlalchemy.engine.base.Engine {}
2017-04-23 16:10:35,900 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-04-23 16:10:35,904 INFO sqlalchemy.engine.base.Engine {}
2017-04-23 16:10:35,957 INFO sqlalchemy.engine.base.Engine 
DROP TABLE station
2017-04-23 16:10:35,959 INFO sqlalchemy.engine.base.Engine {}
2017-04-23 16:10:36,096 INFO sqlalchemy.engine.base.Engine COMMIT
2017-04-23 16:10:36,138 INFO sqlalchemy.engine.base.Engine 
DROP TABLE availability
2017-04-23 16:10:36,140 INFO sqlalchemy.engine.base.Engine {}


In [ ]:
#!/usr/bin/env python
import requests
import traceback
import datetime
import time
import pprint

APIKEY="d19d0fe681cca0b47b75d1cf976d548282e87c7e"
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = None
        
def write_to_file(text,now):
    if not os.path.exists('data'):
        os.makedirs('data')
    with open("data/bikes_{}".format(now).replace(" ", "_"), "w") as f:
           f.write(r.text)

def write_to_db(text):
    try:
        if isinstance(text, str):
            res = json.loads(text)
            engine.execute(availability.insert(), res)
    except Exception as e:
        pprint.pprint(e)
    return

def stations_to_db(text):
    pprint.pprint(text)
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get('address'), int(station.get('banking')), station.get('bike_stands'), int(station.get('bonus')), 
                       station.get('contract_name'), station.get('name'), station.get('number'), 
                       station.get('position').get('lat'), station.get('position').get('lng'), station.get('status')
                       )
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", 
                       vals)
def main():
        while True:
            try:
                now = datetime.datetime.now()
                global r
                r = requests.get(STATIONS, params={"apiKey": APIKEY, "contract": NAME})
                print(r, now)
                write_to_file(r.text,now)                
                write_to_db(r.text)
                stations_to_db(r.text)
                time.sleep(5*60)
            except:
                print(traceback.format_exc())
                if engine is None:

                    return
main()

<Response [200]> 2017-04-23 16:10:36.588751
2017-04-23 16:10:36,932 INFO sqlalchemy.engine.base.Engine INSERT INTO availability (available_bikes, available_bike_stands, number, last_update) VALUES (%(available_bikes)s, %(available_bike_stands)s, %(number)s, %(last_update)s)
2017-04-23 16:10:36,933 INFO sqlalchemy.engine.base.Engine ({'available_bike_stands': 26, 'number': 42, 'available_bikes': 3, 'last_update': 1492960086000}, {'available_bike_stands': 20, 'number': 30, 'available_bikes': 0, 'last_update': 1492959708000}, {'available_bike_stands': 11, 'number': 32, 'available_bikes': 18, 'last_update': 1492959862000}, {'available_bike_stands': 34, 'number': 48, 'available_bikes': 6, 'last_update': 1492960154000}, {'available_bike_stands': 13, 'number': 13, 'available_bikes': 17, 'last_update': 1492959848000}, {'available_bike_stands': 37, 'number': 81, 'available_bikes': 3, 'last_update': 1492960026000}, {'available_bike_stands': 40, 'number': 68, 'available_bikes': 0, 'last_update': 

In [ ]:
def stations_fix_keys(station):
    station['position_lat'] = station['latitude']
    station['position_lng'] = station['longitude']
    return station

stations = json.loads(open('/Users/Daniel/Downloads/stations.json', 'r').read())

engine.execute(station.insert(), *map(stations_fix_keys, stations))

In [ ]:
def write_to_db(text):
    if isinstance(text, str):
        res = json.loads(text)
    engine.execute(availability.insert(), res)
    return